In [145]:
###Merging statistics data into one dataet

# Open pop dataset, flood, avalanche
# Change column names on each
# Join columns from flood to pop
# Join columns from avalanche to pop
# Save as new tables in db

# New columns:
#pop_sum
#area_km2
#sett_count
#build_count
#
#pop
#km2
#sett
#build
#
#pop_av
#km2_av
#sett_av
#build_av
#
#pop_f029 / f121 / f271
#km2_f029 / f121 / f271
#sett_f029 / f121 / f271
#build_f029 / f121 / f271

In [146]:
import json
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 # required for exporting to postgis
import rioxarray as rxr
from rasterio.crs import CRS
from sqlalchemy import create_engine
import rasterstats
from shapely.ops import transform
from datetime import datetime
import numpy as np

In [147]:
# Load database configuration from file
with open(r'D:\iMMAP\code\db_config\hsdc_local_db_config.json', 'r') as f:
    config = json.load(f)

# Create database URL with credentials
db_url = f"postgresql://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"

# Connect to the database
con = create_engine(db_url)

In [148]:
pd.set_option('display.max_columns', None)

In [149]:
group_by_parameter = 'reg_pcode' #adm2_pcode, adm1_pcode, reg_pcode
table_name = 'afg_admbnda_region' #afg_admbnda_adm2, afg_admbnda_adm1, afg_admbnda_region

In [150]:
pop = gpd.GeoDataFrame.from_postgis('SELECT * from {}_pop_stats'.format(table_name), con)
av = gpd.GeoDataFrame.from_postgis('SELECT * from {}_av_stats'.format(table_name), con)
flood = gpd.GeoDataFrame.from_postgis('SELECT * from {}_flood_stats'.format(table_name), con)

In [151]:
# Renaming pop
pop = pop.rename(columns={'pop_sum': 'pop', 'area_km2': 'km2', 'sett_count': 'sett', 'build_count': 'build'})
pop = pop.drop('people_building', axis=1)

In [152]:
# Renaming + filtering avalanche
av = av.rename(columns={'pop_sum': 'pop_av', 'km2': 'km2_av', 'build': 'build_av'})
av = pd.DataFrame(av[[
        group_by_parameter,
        'km2_av',
        'pop_av',
        'build_av']]
        )

In [153]:
# Renaming + filtering flood
flood = flood.rename(columns={
    
    'pop_sum_029cm': 'pop_f029', 
    'km2_029cm':     'km2_f029', 
    'build_029cm':   'build_f029',
    
    'pop_sum_121cm': 'pop_f121', 
    'km2_121cm':     'km2_f121', 
    'build_121cm':   'build_f121',
    
    'pop_sum_271cm': 'pop_f271', 
    'km2_271cm':     'km2_f271', 
    'build_271cm':   'build_f271'
})

flood = pd.DataFrame(flood[[
        group_by_parameter,
        'pop_f029', 
        'km2_f029', 
        'build_f029',
    
        'pop_f121', 
        'km2_f121', 
        'build_f121',

        'pop_f271', 
        'km2_f271', 
        'build_f271']]
        )

In [154]:
combined_stats = pop.merge(
    flood, 
    on = group_by_parameter, 
    how = 'left').merge(
    av, 
    on = group_by_parameter, 
    how = 'left')


In [155]:
combined_stats = combined_stats.explode()
combined_stats

C:\Users\VMO\AppData\Local\Temp\ipykernel_20428\3745913388.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  combined_stats = combined_stats.explode()


,,id,reg_en,reg_da,reg_pcode,reg_ref,regalt1en,regalt2en,regalt1da,regalt2da,adm0_en,adm0_da,adm0_pcode,date,validon,validto,shape_leng,shape_area,pop,km2,sett,build,pop_density,pop_f029,km2_f029,build_f029,pop_f121,km2_f121,build_f121,pop_f271,km2_f271,build_f271,km2_av,pop_av,build_av,geom
0,0,1,Northern,سهیلی حوزه,NR,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.678772,7.751351,5.235171e+06,77306.891436,4714,1112341,67.719327,6.411514e+05,6401.161821,226165,3.523827e+05,3833.592955,109747,123184.515625,1318.613782,33472,2982.313422,95657.328125,18817,"POLYGON ((68.43099 36.07082, 68.42871 36.06806..."
1,0,2,Central Highland,لوړه مرکزی حوزه,CH,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.035766,3.100092,1.224738e+06,31641.503203,4001,141838,38.706707,4.839781e+04,857.810869,12470,3.501955e+04,602.577511,7939,16340.505859,246.778151,3357,3382.763312,116303.851562,17630,"POLYGON ((65.61898 33.31015, 65.61822 33.30971..."
2,0,3,South Eastern,جنوب ختیځه حوزه,SE,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.949378,4.825809,3.572213e+06,50019.504349,6882,596874,71.416400,6.275626e+05,6432.821280,148378,3.774576e+05,3987.602169,73486,129614.062500,1328.427350,15912,1213.008119,78842.492188,3522,"POLYGON ((68.17123 31.83539, 68.17001 31.83832..."
3,0,4,Southern,جنوبی حوزه,SR,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,27.822820,17.576351,4.322542e+06,185698.953830,7815,1190168,23.277147,1.122232e+06,32422.423859,362885,6.817935e+05,16703.330456,197798,268623.000000,5295.097698,60643,3774.488543,85639.914062,4739,"POLYGON ((67.58714 31.50558, 67.58714 31.50522..."
4,0,5,Eastern,ختیځه حوزه,ER,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,10.390809,2.472625,3.847088e+06,25074.132283,3555,488660,153.428540,5.705485e+05,1354.146253,85563,3.336445e+05,887.750631,39404,172397.265625,390.392365,15698,2542.397698,126115.812500,3832,"POLYGON ((70.00272 34.04593, 70.00179 34.04587..."
5,0,6,Capital,مرکزی حوزه,CR,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,14.491308,3.054274,9.377611e+06,31086.145907,6007,1023867,301.665271,2.286357e+06,1739.510514,245411,1.570888e+06,1048.597876,146535,795946.625000,415.946234,66473,2986.340265,305379.625000,26367,"POLYGON ((67.27185 33.97512, 67.26954 33.97330..."
6,0,7,North Eastern,سهیل ختیځه حوزه,NE,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,27.870669,8.141572,5.463034e+06,80700.710410,6053,995154,67.694989,6.804306e+05,4765.257053,169647,4.464008e+05,3109.974298,86610,207903.015625,1227.975954,28374,10800.063145,402390.718750,33810,"POLYGON ((71.19470 36.03890, 71.19362 36.03821..."
7,0,8,Western,لویدیځه حوزه,WR,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,26.348523,15.642130,5.779767e+06,160616.853965,7817,1038744,35.984810,1.078627e+06,19018.181804,256281,6.499727e+05,11002.056309,132366,262553.687500,3860.510701,34453,6070.830917,117627.031250,20149,"POLYGON ((61.61755 31.38748, 61.60269 31.38935..."


In [156]:
# Save to db
combined_stats.to_postgis('{}_stats'.format(table_name), con, if_exists='replace')

In [157]:
##combined_stats.explode()['adm2_pcode'].duplicated()
#pd.set_option('display.max_rows', None)
#combined_stats.explode().duplicated(subset=['adm2_pcode'])
#
#duplicateRows = combined_stats.explode()[combined_stats.explode().duplicated()]
#duplicateRows

C:\Users\VMO\AppData\Local\Temp\ipykernel_20428\3111013489.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  combined_stats.explode().duplicated(subset=['adm2_pcode'])


KeyError: Index(['adm2_pcode'], dtype='object')